In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import time
import os
from functools import partial
from multiprocessing import Pool

from fastdtw import fastdtw, dtw
from sklearn import svm
from sklearn.metrics import roc_auc_score, roc_curve, accuracy_score

In [2]:
reduced = pd.read_csv("reduced.csv", index_col="date")
d = reduced.apply(np.log).diff()[1:]
data = []
for podatak in d["NKE"]:
    data.append(podatak)
d.shape

(969, 368)

In [40]:
L = len(data)
l = 20
s = 1

X = np.stack([data[i:i + l] for i in range(L // 2 - l - s)])
Y = np.stack([data[i + l + s - 1] > data[i + l - 1] for i in range(L // 2 - l - s)]).astype(np.float32)
##ako je sljedeći element veći od zadnjeg clana niza onda dobiva 1 inace 0

X_test = np.stack([data[i:i + l] for i in range(L // 2, L - l - s)])
Y_test = np.stack([data[i + l + s - 1] > data[i + l - 1] for i in range(L // 2, L - l - s)]).astype(np.float32)
print(X)
print(len(X))
print(len(X_test))

[[-3.97425211e-03 -1.65425759e-02  6.61773055e-03 ...  7.57030272e-03
   2.15881400e-02 -2.14276907e-02]
 [-1.65425759e-02  6.61773055e-03  9.44687195e-03 ...  2.15881400e-02
  -2.14276907e-02  4.64186508e-03]
 [ 6.61773055e-03  9.44687195e-03  4.61086754e-03 ... -2.14276907e-02
   4.64186508e-03  3.82531547e-03]
 ...
 [-7.80472243e-03  1.10436234e-03  7.02141561e-04 ... -9.90638467e-05
  -1.07570758e-02  3.09953755e-03]
 [ 1.10436234e-03  7.02141561e-04 -1.20397326e-03 ... -1.07570758e-02
   3.09953755e-03 -1.33663615e-02]
 [ 7.02141561e-04 -1.20397326e-03  1.22725412e-02 ...  3.09953755e-03
  -1.33663615e-02  1.94375421e-02]]
463
464


In [21]:
L = len(data)
l = 20
s = 1

X = np.stack([data[i:i + l] for i in range(2*L // 3 - l - s)])
Y = np.stack([data[i + l + s - 1] > data[i + l - 1] for i in range(2*L // 3 - l - s)]).astype(np.float32)
##ako je sljedeći element veći od zadnjeg clana niza onda dobiva 1 inace 0

X_test = np.stack([data[i:i + l] for i in range(2*L // 3, L - l - s)])
Y_test = np.stack([data[i + l + s - 1] > data[i + l - 1] for i in range(2*L // 3, L - l - s)]).astype(np.float32)
print(len(X))
print(len(X_test))

625
302


In [3]:
L = len(data)
l = 20
s = 1

X = np.stack([data[i:i + l] for i in range(3*L // 4 - l - s)])
Y = np.stack([data[i + l + s - 1] > data[i + l - 1] for i in range(3*L // 4 - l - s)]).astype(np.float32)
##ako je sljedeći element veći od zadnjeg clana niza onda dobiva 1 inace 0

X_test = np.stack([data[i:i + l] for i in range(3*L // 4, L - l - s)])
Y_test = np.stack([data[i + l + s - 1] > data[i + l - 1] for i in range(3*L // 4, L - l - s)]).astype(np.float32)
print(len(X))
print(len(X_test))

705
222


In [4]:
len(X), len(X_test), Y.mean(), Y_test.mean()

(705, 222, 0.4992908, 0.518018)

In [5]:
n = len(X)
m = len(X_test)
kernel_train = np.empty((n, n))
kernel_test = np.empty((m, n))

In [6]:
t0 = time.time()
for i in range(n):
    for j in range(i+1):
        if i == j:
            kernel_train[i][j] = 0.0
        else:
            kernel_train[i][j] = kernel_train[j][i] = fastdtw(X[i], X[j], radius = 5, dist = 2)[0]


for i in range(m):
    for j in range(n):           
        kernel_test[i][j] = fastdtw(X_test[i], X[j], radius = 5, dist = 2)[0]
                      
print(time.time()-t0)

KeyboardInterrupt: 

In [7]:
#np.save("kernel_train_univariant_NKE.npy", kernel_train)

In [25]:
#np.save("kernel_train_univariant_NKE_NEW.npy", kernel_train)

In [ ]:
np.save("kernel_train_univariant_NKE_NEW_3/4.npy", kernel_train)

In [37]:
#kernel_train = np.load("kernel_train_univariant_NKE_NEW.npy")

In [41]:
#kernel_train = np.load("kernel_train_univariant_NKE.npy")

In [ ]:
kernel_train = np.load("kernel_train_univariant_NKE_NEW_3/4.npy")

In [9]:
#np.save("kernel_test_univariant_NKE.npy", kernel_test)

In [27]:
#np.save("kernel_test_univariant_NKE_NEW.npy", kernel_test)

In [ ]:
np.save("kernel_test_univariant_NKE_NEW_3/4.npy", kernel_test)

In [38]:
#kernel_test = np.load("kernel_test_univariant_NKE_NEW.npy")

In [42]:
#kernel_test = np.load("kernel_test_univariant_NKE.npy")

In [ ]:
kernel_test = np.load("kernel_test_univariant_NKE_NEW_3/4.npy")

In [43]:
c = 10
gamma = 25
kernel = np.exp(-kernel_train/gamma)
testkernel = np.exp(-kernel_test/gamma)
clf = svm.SVC(kernel='precomputed', C=c)
clf.fit(kernel, Y)
res = clf.predict(kernel)
#train set
print('accuracy train: ', np.mean(res == Y))
print('Area under curve(AUC) train:', roc_auc_score(Y, res))
#test set
res = clf.predict(testkernel)
print('accuracy test: ', np.mean(res == Y_test))
print('Area under curve(AUC) test:', roc_auc_score(Y_test, res))

accuracy train:  0.7473002159827213
Area under curve(AUC) train: 0.7471076693412949
accuracy test:  0.7349137931034483
Area under curve(AUC) test: 0.7347234611953612
